In [2]:
import requests
from bs4 import BeautifulSoup as BS
import pandas as pd

In [5]:
url = 'https://www.billboard.com/charts/hot-100/'

response = requests.get(url)

print(type(response))
print(response.status_code)

print(requests.get('https://www.billboard.com/charts/hot-100/'))

soup = BS(response.text)

<class 'requests.models.Response'>
200
<Response [200]>
